* **Submission of Assignment 7**
* **Lakshay Singla**
* **102003371**
* **3CO15**

## **Question 1**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/ham_spam.csv', encoding='latin-1')
data = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace = True)
df.rename(columns = {"v2":"text", "v1":"output"}, inplace = True)
df.head(2)

,output,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [ ]:
import nltk
from nltk import sent_tokenize,word_tokenize
import re
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Preprocessing**

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
stopword = stopwords.words('english')

In [ ]:
for i in range(len(df)):
  text = df.iloc[i][1]
  text = word_tokenize(text)
  text.append(" playing")
  new_text = [ps.stem(word.lower()) for word in text if word.lower() not in stopword and word.isalpha()]
  new_text = ' '.join(new_text)
  df.iloc[i][1] = new_text

In [ ]:
df.head(2)

,output,text
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni


**Making the TDM**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)

In [ ]:
X = cv.fit_transform(df['text']).toarray()
X.shape

(5572, 1000)

In [ ]:
tdm = pd.DataFrame(X, columns = cv.get_feature_names_out())
tdm.head(3)

,abiola,abl,abt,accept,access,account,across,activ,actual,ad,...,ye,yeah,year,yep,yesterday,yet,yo,yr,yup,ìï
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Train Test Splitting**

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
y = LE.fit_transform(df['output'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tdm, y, train_size = 0.7, random_state = 43)

**Applying Naive-Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm


array([[1144,  296],
       [  20,  212]])

## **Question 2**

In [9]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [10]:
x = iris.data
y = iris.target

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 43)

**From Scratch**

**Output_Label Probabilities**

In [5]:
import numpy as np
size = x_train.shape[0]
label_prob = {}
for outcome in np.unique(y_train):
  outcome_count = sum(outcome==y_train)
  label_prob[outcome] = outcome_count/size
print(label_prob)

{0: 0.3333333333333333, 1: 0.3142857142857143, 2: 0.3523809523809524}


**Computing Liklihood**

In [33]:
target_names = list(iris.target_names)
target_names

['setosa', 'versicolor', 'virginica']

In [110]:
df = pd.DataFrame(x_train,columns = iris.feature_names)
df['target'] = y_train
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.7,2.8,4.5,1.3,1
1,5.5,2.6,4.4,1.2,1
2,5.7,2.6,3.5,1.0,1
3,6.7,3.1,4.4,1.4,1
4,6.3,3.3,4.7,1.6,1


**Calculate Gaussian Prob**

In [127]:
def GaussianProb(mean,var,new_x):
  exp = np.exp((-(new_x - mean)**2)/(2*var))
  start = (1/(np.sqrt(2*np.pi*var))) 
  return start*exp

**Making mean and variance table**

In [128]:
final_table = []
for i in range(3):
  new_table = []
  temp_df = df[df['target'] == i]
  for feature in iris.feature_names:
    mean,var = temp_df[feature].mean() , temp_df[feature].var()
    new_table.append([mean,var])
  final_table.append(new_table)

In [129]:
final_table[0]

[[5.031428571428571, 0.13457142857142856],
 [3.4342857142857146, 0.14467226890756302],
 [1.4828571428571429, 0.03322689075630252],
 [0.2571428571428571, 0.013109243697478993]]

**Predicting output**

In [130]:
test = x_test[0]
test

array([4.8, 3.1, 1.6, 0.2])

In [146]:
x_train[:5]

array([[5.7, 2.8, 4.5, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5.7, 2.6, 3.5, 1. ],
       [6.7, 3.1, 4.4, 1.4],
       [6.3, 3.3, 4.7, 1.6]])

In [162]:
y_pred = []
for i in range(len(x_test)): 
  test = x_test[i]
  probs = {}
  count = 0
  for each_label in final_table:  
    prob = label_prob[count]  
    feature_count = 0 
    for stats in each_label:  
     prob = prob * GaussianProb(stats[0],stats[1],test[feature_count])
     feature_count+=1 
    count+=1  
    probs[count-1] = prob 
  output =max(probs, key=probs.get)
  y_pred.append(output)

In [ ]:
for i in range(len(y_test)):
  print(y_pred[i], y_test[i])

In [169]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.94      0.97        17
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



**USING INBUILT**

In [171]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[15,  0,  0],
       [ 0, 16,  1],
       [ 0,  0, 13]])

In [172]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.94      0.97        17
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



## **Question 3**

**Loading the iris dataset**

In [164]:
from sklearn import datasets
iris = datasets.load_iris()

**Setting classifer**

In [165]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [166]:
knn = KNeighborsClassifier()
params = {
    'n_neighbors' : [1,2,3,4,5],
     'weights' : ['uniform', 'distance'],
     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
     'p' : [1,2]
}

**Using GridSearchCV**

In [167]:
classifier = GridSearchCV(estimator = knn, param_grid = params)
classifier.fit(iris.data,iris.target)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 3, 4, 5], 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [168]:
print(classifier.best_params_)
print(classifier.best_score_)

{'algorithm': 'auto', 'n_neighbors': 4, 'p': 2, 'weights': 'uniform'}
0.9733333333333334


## **Submitted by Lakshay Singla**
**_____________________ _____________________ _____________________**